In [1]:
!pip install --no-index --find-links /kaggle/input/cps-env/wheels/wheels --no-deps cellpose fastremap fill_voids roifile

Looking in links: /kaggle/input/cps-env/wheels/wheels
Processing /kaggle/input/cps-env/wheels/wheels/cellpose-4.0.4.dev7+gfb5a6c0-py3-none-any.whl
Processing /kaggle/input/cps-env/wheels/wheels/fastremap-1.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/cps-env/wheels/wheels/fill_voids-2.0.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/cps-env/wheels/wheels/roifile-2025.5.10-py3-none-any.whl


In [ ]:
from cellpose import models, io
import numpy as np
import pandas as pd
from pathlib import Path
import cv2


ensemble_weights = [
    #'/kaggle/input/patch4_base_bs2/pytorch/default/1/patch_epoch100_bs2_epoch_0031',
    '/kaggle/input/cosbest/pytorch/default/3/patch_epoch100_bs2__epoch_0031',#0.339
    #'/kaggle/input/cosbest/pytorch/default/1/patch_epoch100_bs2__epoch_0031',
    # '/kaggle/input/cellpose-fold-weight/fold0_epoch_0032_mAP_0.3129.pth',#0.334
    # '/kaggle/input/cellpose-fold-weight/fold1_epoch_0038_mAP_0.2929.pth',#0.335
    # '/kaggle/input/cellpose-fold-weight/fold2_epoch_0058_mAP_0.3104.pth', #0.334
    # '/kaggle/input/fold3-best/pytorch/default/1/fullres_fold3_epoch_0067', #0.336
    # '/kaggle/input/cellpose-fold-weight/fold4_epoch_0047_mAP_0.2945.pth'#0.335
    
]


models_list = [
    models.CellposeModel(
        gpu=True,
        pretrained_model=weight_path
    )
    for weight_path in ensemble_weights
]

print(f"Loaded {len(models_list)} models for ensemble.")



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4.dev7+gfb5a6c0 
platform:       	linux 
python version: 	3.11.11 
torch version:  	2.6.0+cu124! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


Loaded 1 models for ensemble.


In [3]:
import numpy as np

def rle_encode(mask):
    """
    Encode a binary mask into RLE string.
    mask: 2D numpy array, 1=foreground, 0=background
    returns: RLE string ("start length" format)
    """
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(rle_str, shape=(520, 704)):
    """
    Decode RLE string back to a binary mask.
    rle_str: RLE string ("start length")
    shape: (height, width) of output mask
    returns: 2D numpy array mask
    """
    if not isinstance(rle_str, str) or not rle_str:
        return np.zeros(shape, dtype=np.uint8)
    parts = rle_str.split()
    starts = np.asarray(parts[0::2], dtype=int) - 1
    lengths = np.asarray(parts[1::2], dtype=int)
    mask_flat = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for s, l in zip(starts, lengths):
        mask_flat[s:s + l] = 1
    return mask_flat.reshape(shape)

def compute_iou(mask1, mask2):
    """
    Compute IoU of two binary masks.
    mask1, mask2: 2D arrays (bool or uint8)
    returns: IoU float
    """
    m1 = mask1.astype(bool)
    m2 = mask2.astype(bool)
    inter = np.logical_and(m1, m2).sum()
    union = np.logical_or(m1, m2).sum()
    return inter / union if union > 0 else 0.0


In [4]:
import pycocotools.mask
import torch
import numpy as np
import cv2

def maskwise_nms(masks, iou_thresh=0.6):
    """
    Perform mask-level Non-Maximum Suppression (NMS).
    masks: np.ndarray of shape (K, H, W) containing binary masks
    iou_thresh: IoU threshold for suppression
    returns: np.ndarray of shape (M, H, W) with NMS‐filtered masks
    """
    keep_indices = []
    K = masks.shape[0]
    for i in range(K):
        if i in keep_indices:
            continue
        m1 = masks[i].astype(bool)
        suppress = False
        for j in keep_indices:
            m2 = masks[j].astype(bool)
            # compute IoU
            inter = np.logical_and(m1, m2).sum()
            union = np.logical_or(m1, m2).sum()
            iou = inter / union if union > 0 else 0.0
            if iou > iou_thresh:
                suppress = True
                break
        if not suppress:
            keep_indices.append(i)
    if not keep_indices:
        return np.zeros((0, masks.shape[1], masks.shape[2]), dtype=np.uint8)
    return masks[keep_indices]


def instmap_to_masks_boxes(inst_map):
    """
    Convert a 2D instance label map into binary masks and bounding boxes.
    
    Args:
        inst_map (np.ndarray): shape = (H, W), dense instance map
                              0 = background, 1, 2, 3... = instance labels.
    
    Returns:
        masks (np.ndarray): shape = (K, H, W) of binary masks.
        boxes (np.ndarray): shape = (K, 6), each row = [x0, y0, x1, y1, area, 0.0].
                            Returns (None, None) if no instances found.
    """
    masks, boxes = [], []
    for lab in np.unique(inst_map)[1:]:
        m = (inst_map == lab).astype(np.uint8)
        ys, xs = np.where(m)
        if ys.size == 0:
            continue
        area = int(m.sum())
        masks.append(m)
        boxes.append([xs.min(), ys.min(), xs.max(), ys.max(), area, 0.0])
    if not masks:
        return None, None
    return np.stack(masks, axis=0), np.array(boxes, dtype=float)


def weighted_mask_fusion_nmw(masks, boxes, scores, iou_thr=0.15, score_coef=0.8):
    """
    Weighted NMW (Non-Maximum Weighted) fusion:
      1. Compute pairwise IoU for all masks -> ious (N×N).
      2. Treat masks with IoU > iou_thr as the same cluster.
      3. For each cluster, multiply each mask's score by score_coef, then normalize -> weights.
      4. Compute pixel-wise weighted sum across masks in cluster -> soft_map.
      5. Threshold soft_map at 0.5 -> fused mask; if all < 0.5, pick the highest-score original mask.

    Args:
        masks (np.ndarray): shape = (N, H, W), list of binary masks.
        boxes (np.ndarray): shape = (N, 6), each row = [x0, y0, x1, y1, area, dummy_score].
        scores (np.ndarray): shape = (N,), float scores for each mask.
        iou_thr (float): IoU threshold to form clusters.
        score_coef (float): coefficient < 1 to scale scores before normalization.

    Returns:
        fused_masks (np.ndarray): shape = (M, H, W), list of fused binary masks.
        fused_boxes (np.ndarray): shape = (M, 6), list of bounding boxes for fused masks.
    """
    N = masks.shape[0]
    if N == 0:
        return (
            np.zeros((0, masks.shape[1], masks.shape[2]), dtype=np.uint8),
            np.zeros((0, 6), dtype=float),
        )

    rles = [pycocotools.mask.encode(np.asfortranarray(m)) for m in masks]
    ious = pycocotools.mask.iou(rles, rles, [0] * N)
    used = set()
    fused_masks, fused_boxes = [], []

    for i in range(N):
        if i in used:
            continue
        group = [i]
        for j in range(i + 1, N):
            if ious[i, j] > iou_thr:
                group.append(j)
        used.update(group)

        sub_masks = masks[group]
        sub_boxes = boxes[group]
        sub_scores = scores[group].astype(float) * score_coef

        if len(group) == 1:
            fused_masks.append(sub_masks[0])
            fused_boxes.append(sub_boxes[0])
            continue

        weights = sub_scores / sub_scores.sum()
        soft_map = np.tensordot(weights, sub_masks, axes=(0, 0))
        bin_mask = (soft_map >= 0.5).astype(np.uint8)

        ys, xs = np.where(bin_mask)
        if ys.size == 0:
            best_idx = group[np.argmax(sub_scores)]
            bin_mask = masks[best_idx]
            ys, xs = np.where(bin_mask)
            fused_masks.append(bin_mask)
            fused_boxes.append([
                xs.min(), ys.min(), xs.max(), ys.max(),
                int(bin_mask.sum()), 0.0
            ])
        else:
            fused_masks.append(bin_mask)
            fused_boxes.append([
                xs.min(), ys.min(), xs.max(), ys.max(),
                int(bin_mask.sum()), 0.0
            ])

    if not fused_masks:
        return (
            np.zeros((0, masks.shape[1], masks.shape[2]), dtype=np.uint8),
            np.zeros((0, 6), dtype=float),
        )
    return np.stack(fused_masks, axis=0), np.array(fused_boxes, dtype=float)


In [5]:
import numpy as np
import cv2


def tta_predict_probability_and_flows(model, image, image_id):
    """
    Use TTA (original + horizontal flip) to compute averaged probability and flow maps.

    Args:
        model: CellposeModel instance.
        image: numpy array of shape (H, W, 3).
        image_id: string identifier (not used here).

    Returns:
        averaged_prob_map: numpy array of shape (H, W), dtype float32.
        averaged_dp_map: numpy array of shape (2, H, W), dtype float32.
    """
    H, W = image.shape[:2]

    # Two augmentations: original and horizontally flipped
    aug_images = [
        image,
        np.flip(image, axis=1),
    ]

    # Inverse transforms to revert flips
    inverse_ops_prob = [
        lambda x: x,
        lambda x: np.flip(x, axis=1),
    ]
    inverse_ops_dp = [
        lambda dp: dp,
        lambda dp: np.stack([
            np.flip(dp[0], axis=1),
            -np.flip(dp[1], axis=1),
        ], axis=0),
    ]

    all_prob_maps = []
    all_dp_maps = []

    for aug_idx, aug_img in enumerate(aug_images):
        # Run model.eval on augmented image (no mask decoding)
        _, eval_flows, _ = model.eval(aug_img, compute_masks=False)
        pred_dp_map_raw = eval_flows[1].astype(np.float32)   # shape = (2, h', w')
        pred_prob_map_raw = eval_flows[2].astype(np.float32) # shape = (h', w')

        # Apply inverse transform to flip results back
        prob_aug = inverse_ops_prob[aug_idx](pred_prob_map_raw)
        dp_aug = inverse_ops_dp[aug_idx](pred_dp_map_raw)

        # Resize probability map if shape differs
        if prob_aug.shape[:2] != (H, W):
            prob_aug = cv2.resize(
                prob_aug, (W, H), interpolation=cv2.INTER_LINEAR
            )

        # Resize flow map if shape differs
        if dp_aug.shape[1:] != (H, W):
            dp_x = cv2.resize(
                dp_aug[0], (W, H), interpolation=cv2.INTER_LINEAR
            )
            dp_y = cv2.resize(
                dp_aug[1], (W, H), interpolation=cv2.INTER_LINEAR
            )
            dp_aug = np.stack([dp_x, dp_y], axis=0)

        all_prob_maps.append(prob_aug)
        all_dp_maps.append(dp_aug)

    # Compute average over augmentations
    averaged_prob_map = np.mean(
        np.stack(all_prob_maps, axis=0), axis=0
    )
    averaged_dp_map = np.mean(
        np.stack(all_dp_maps, axis=0), axis=0
    )

    return averaged_prob_map, averaged_dp_map


In [6]:
# Inference + Ensemble (multi-model TTA) + NMW post-processing
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import pandas as pd
from pathlib import Path
import skimage.io as io
from cellpose import dynamics, models, core, io as cpio, metrics
import cv2

# parameters configs
decode_flow_th = 0.4
iou_thr_nmw = 0.45
score_coef = 0.8
min_size = 75
# corrupt = True
# cell_type = 0

test_dir = Path('/kaggle/input/sartorius-cell-instance-segmentation/test')
test_files = sorted(f for f in test_dir.iterdir() if f.suffix == '.png')

submission_data = []
for img_path in tqdm(test_files, desc="Inference"):
    img_id = img_path.stem
    img = io.imread(str(img_path))
    H, W = img.shape[:2]

    all_masks, all_boxes, all_scores = [], [], []
    for mdl in models_list:
        # TTA -> get avg_prob_map, avg_dp_map
        avg_prob_map, avg_dp_map = tta_predict_probability_and_flows(mdl, img, "")
        # decode dense instance map
        combined_mask = dynamics.resize_and_compute_masks(
            avg_dp_map, avg_prob_map,
            cellprob_threshold=0.0, flow_threshold=decode_flow_th,
            min_size=20, resize=(H, W), device=mdl.device
        )
        if combined_mask.max() == 0:
            continue

        m_bin, bxs = instmap_to_masks_boxes(combined_mask)
        if m_bin is None:
            continue

        # compute instance scores
        inst_scores = []
        for mask_k in m_bin:
            mbool = mask_k.astype(bool)
            inst_scores.append(float(avg_prob_map[mbool].mean()) if mbool.sum() > 0 else 0.0)
        inst_scores = np.array(inst_scores)

        all_masks.append(m_bin)
        all_boxes.append(bxs)
        all_scores.append(inst_scores)

    if not all_masks:
        submission_data.append({'id': img_id, 'predicted': ''})
        continue

    masks_concat = np.concatenate(all_masks, axis=0)
    boxes_concat = np.concatenate(all_boxes, axis=0)
    scores_concat = np.concatenate(all_scores, axis=0)

    # Weighted Mask Fusion (NMW)
    fused_masks, fused_boxes = weighted_mask_fusion_nmw(
        masks_concat, boxes_concat, scores_concat,
        iou_thr=iou_thr_nmw, score_coef=score_coef
    )

    # filter small masks
    if fused_masks.shape[0] > 0:
        areas = fused_masks.sum(axis=(1, 2))
        keep = np.where(areas > min_size)[0]
        fused_masks = fused_masks[keep]
        fused_boxes = fused_boxes[keep]

    if fused_masks.shape[0] > 0:
        fused_masks = maskwise_nms(fused_masks, iou_thresh=0.6)


    
    # reconstruct final instance map & encode to RLE
    canvas = np.zeros((H, W), dtype=np.int32)
    for m in fused_masks:
        canvas[m.astype(bool)] = canvas.max() + 1

    if canvas.max() > 0:
        for inst_id in np.unique(canvas)[1:]:
            binary_mask = (canvas == inst_id).astype(np.uint8)
            rle = rle_encode(binary_mask)
            submission_data.append({'id': img_id, 'predicted': rle})
    else:
        submission_data.append({'id': img_id, 'predicted': ''})

# assemble submission DataFrame
submission_df = pd.DataFrame(submission_data, columns=['id', 'predicted'])
all_test_ids = [f.stem for f in test_files]
missing_ids = set(all_test_ids) - set(submission_df['id'].unique())
for img_id in missing_ids:
    submission_df = pd.concat(
        [submission_df, pd.DataFrame([{'id': img_id, 'predicted': ''}])],
        ignore_index=True
    )
submission_df = submission_df.sort_values(by='id').reset_index(drop=True)

# check duplicates
if submission_df.duplicated(subset=['id'], keep=False).any():
    print("There are more than one instance in the images.")

# check pixel overlaps
rle_dict = defaultdict(list)
for img_id, rle_text in zip(submission_df['id'], submission_df['predicted']):
    if isinstance(rle_text, str) and rle_text.strip():
        rle_dict[img_id].append(rle_text)

overlap_count = 0
for img_id, rles in tqdm(rle_dict.items(), desc="Checking pixel overlaps"):
    mask_sum = np.zeros((520, 704), dtype=np.uint8)
    for rle_text in rles:
        mask = rle_decode(rle_text, (520, 704))
        mask_sum += mask
    if (mask_sum > 1).any():
        print(f"Image {img_id} has overlapping pixels.")
        overlap_count += 1

if overlap_count == 0:
    print("All images have no overlapping pixels.")
else:
    print(f"{overlap_count} images have overlapping issues, please review.")

submission_df.to_csv('submission.csv', index=False)

Inference: 100%|██████████| 3/3 [00:24<00:00,  8.08s/it]


There are more than one instance in the images.


Checking pixel overlaps: 100%|██████████| 3/3 [00:00<00:00, 153.98it/s]

All images have no overlapping pixels.
